# Addition Task Implementation Walkthrough

This notebook demonstrates the step-by-step implementation of a flexible multi-digit addition task system for causal abstraction experiments. Addition tasks test how language models perform arithmetic and whether they use interpretable computational structures.

## What are Addition Tasks?

Addition tasks involve:
1. **Input Numbers**: K numbers, each with D digits (e.g., 23 + 45 has K=2, D=2)
2. **Templates**: Structured prompts that ask for sums (e.g., "The sum of 23 and 45 is")
3. **Output**: The correct sum (e.g., 68)

Example: "The sum of 23 and 45 is" → model completes with "68"

The goal is to understand how neural networks compute sums and whether they use human-interpretable structures like carry propagation.

## Step 1: Core Data Structures

First, we need a configuration that defines the task structure.

In [21]:
# Autoreload imports
%load_ext autoreload
%autoreload 2

# Setup imports
import sys

sys.path.append("../..")

from causalab.tasks.general_addition.config import (
    create_two_number_two_digit_config,
    create_two_number_three_digit_config,
    create_three_number_two_digit_config,
    create_general_config,
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### AdditionTaskConfig: The Blueprint

This configuration class defines everything about our addition task:

In [22]:
# Create a 2-number, 2-digit configuration
config_2n_2d = create_two_number_two_digit_config()

print("2-Number, 2-Digit Configuration:")
print(f"Max numbers (K): {config_2n_2d.max_numbers}")
print(f"Max digits (D): {config_2n_2d.max_digits}")
print(f"Templates: {config_2n_2d.templates}")
print('Example prompt structure: "The sum of [10-99] and [10-99] is"')

2-Number, 2-Digit Configuration:
Max numbers (K): 2
Max digits (D): 2
Templates: ['The sum of {num0} and {num1} is']
Example prompt structure: "The sum of [10-99] and [10-99] is"


### Different Task Configurations

The system supports various configurations:

In [23]:
# Create different configurations
config_2n_3d = create_two_number_three_digit_config()
config_3n_2d = create_three_number_two_digit_config()
config_4n_2d = create_general_config(4, 2)

print("Different Task Configurations:")
print()
print("1. Two 2-digit numbers (23 + 45):")
print(f"   Template: {config_2n_2d.templates[0]}")
print("   Example: The sum of 23 and 45 is 68")
print()
print("2. Two 3-digit numbers (123 + 456):")
print(f"   Template: {config_2n_3d.templates[0]}")
print("   Example: The sum of 123 and 456 is 579")
print()
print("3. Three 2-digit numbers (12 + 34 + 56):")
print(f"   Template: {config_3n_2d.templates[0]}")
print("   Example: The sum of 12, 34, and 56 is 102")
print()
print("4. Four 2-digit numbers (12 + 34 + 56 + 78):")
print(f"   Template: {config_4n_2d.templates[0]}")
print("   Example: The sum of 12, 34, 56, and 78 is 180")

Different Task Configurations:

1. Two 2-digit numbers (23 + 45):
   Template: The sum of {num0} and {num1} is
   Example: The sum of 23 and 45 is 68

2. Two 3-digit numbers (123 + 456):
   Template: The sum of {num0} and {num1} is
   Example: The sum of 123 and 456 is 579

3. Three 2-digit numbers (12 + 34 + 56):
   Template: The sum of {num0}, {num1}, and {num2} is
   Example: The sum of 12, 34, and 56 is 102

4. Four 2-digit numbers (12 + 34 + 56 + 78):
   Template: The sum of {num0}, {num1}, {num2}, and {num3} is
   Example: The sum of 12, 34, 56, and 78 is 180


## Step 2: Template Processing System

The template system converts digit values into formatted prompts.

In [24]:
from causalab.tasks.general_addition.templates import AdditionTemplateProcessor

# Create processor for 2-digit addition
processor = AdditionTemplateProcessor(config_2n_2d)

### Understanding Digit Representation

Numbers are represented as lists of digits, with index 0 being the **most significant** (leftmost) digit.

In [25]:
print("Digit Representation Examples:")
print()

# Example 1: 23
digits_23 = [2, 3]
number_23 = processor.digits_to_number(digits_23)
print(f"Digits {digits_23} -> Number {number_23}")

# Example 2: 456
digits_456 = [0, 5, 6]
number_456 = processor.digits_to_number(digits_456)
print(f"Digits {digits_456} -> Number {number_456}")

# Reverse: number to digits
print()
print("Number to Digits (with padding):")
digits_68 = processor.number_to_digits(68, 2)
print(f"Number 68 with 2 digits -> {digits_68}")

digits_068 = processor.number_to_digits(68, 3)
print(f"Number 68 with 3 digits -> {digits_068}")

Digit Representation Examples:

Digits [2, 3] -> Number 23
Digits [0, 5, 6] -> Number 56

Number to Digits (with padding):
Number 68 with 2 digits -> [6, 8]
Number 68 with 3 digits -> [0, 6, 8]


### Template Filling

Templates use placeholders like {num0}, {num1} that get filled with formatted numbers:

In [26]:
# Create some example numbers
number1 = [2, 3]  # 23
number2 = [4, 5]  # 45

# Fill template
template = config_2n_2d.templates[0]
prompt = processor.fill_template(template, [number1, number2])

print("Template Filling:")
print(f"Template: {template}")
print(
    f"Numbers: {number1} ({processor.digits_to_number(number1)}), {number2} ({processor.digits_to_number(number2)})"
)
print(f"Result: '{prompt}'")

Template Filling:
Template: The sum of {num0} and {num1} is
Numbers: [2, 3] (23), [4, 5] (45)
Result: 'The sum of 23 and 45 is'


### Computing Sums

The processor can compute the correct sum:

In [27]:
# Compute sum of 23 + 45
output_digits = processor.compute_sum([number1, number2])
output_str = processor.format_output(output_digits)

print("Sum Computation:")
print(
    f"Input: {processor.digits_to_number(number1)} + {processor.digits_to_number(number2)}"
)
print(f"Output digits: {output_digits}")
print(f"Formatted output: {output_str}")
print(
    f"Verification: {processor.digits_to_number(number1) + processor.digits_to_number(number2)} = {output_str}"
)

Sum Computation:
Input: 23 + 45
Output digits: [0, 6, 8]
Formatted output: 68
Verification: 68 = 68


### Complete Prompt Generation

Putting it all together from digit dictionary:

In [28]:
# Create digit dictionary (the format used by causal models)
digit_dict = {
    "digit_0_0": 2,  # First number, first digit (tens place)
    "digit_0_1": 3,  # First number, second digit (ones place)
    "digit_1_0": 4,  # Second number, first digit
    "digit_1_1": 5,  # Second number, second digit
}

# Generate complete prompt
full_prompt = processor.generate_prompt(digit_dict, num_addends=2, num_digits=2)

print("Complete Prompt Generation:")
print(f"Digit dictionary: {digit_dict}")
print(f"Generated prompt: '{full_prompt}'")
print()
print("This is the text that will be fed to the language model.")

Complete Prompt Generation:
Digit dictionary: {'digit_0_0': 2, 'digit_0_1': 3, 'digit_1_0': 4, 'digit_1_1': 5}
Generated prompt: 'The sum of 23 and 45 is'

This is the text that will be fed to the language model.


### Examples with Different Configurations

In [29]:
# 3-digit addition
processor_3d = AdditionTemplateProcessor(config_2n_3d)
digit_dict_3d = {
    "digit_0_0": 1,
    "digit_0_1": 2,
    "digit_0_2": 3,  # 123
    "digit_1_0": 4,
    "digit_1_1": 5,
    "digit_1_2": 6,  # 456
}
prompt_3d = processor_3d.generate_prompt(digit_dict_3d, num_addends=2, num_digits=3)
print("3-Digit Addition:")
print(f"  Prompt: '{prompt_3d}'")
print("  Expected: 123 + 456 = 579")
print()

# 3-number addition
processor_3n = AdditionTemplateProcessor(config_3n_2d)
digit_dict_3n = {
    "digit_0_0": 1,
    "digit_0_1": 2,  # 12
    "digit_1_0": 3,
    "digit_1_1": 4,  # 34
    "digit_2_0": 5,
    "digit_2_1": 6,  # 56
}
prompt_3n = processor_3n.generate_prompt(digit_dict_3n, num_addends=3, num_digits=2)
print("3-Number Addition:")
print(f"  Prompt: '{prompt_3n}'")
print("  Expected: 12 + 34 + 56 = 102")

3-Digit Addition:
  Prompt: 'The sum of 123 and 456 is'
  Expected: 123 + 456 = 579

3-Number Addition:
  Prompt: 'The sum of 12, 34, and 56 is'
  Expected: 12 + 34 + 56 = 102


## Step 3: Basic Causal Model

Now we integrate everything into a causal model that can generate prompts and compute answers.

In [30]:
from causalab.tasks.general_addition.causal_models import (
    create_basic_addition_model,
    sample_valid_addition_input,
)

### Causal Model Structure

The basic model has:
- **Input variables**: digit_{k}_{d} for each digit in each number
- **Control variables**: num_addends, num_digits, template
- **Output variables**: output_digit_{d} for each digit in the result
- **Special variables**: raw_input (the prompt), raw_output (the answer)

In [31]:
# Create basic causal model for 2-digit addition
basic_model = create_basic_addition_model(config_2n_2d)

print(f"Basic Causal Model: {basic_model.id}")
print(f"Total variables: {len(basic_model.variables)}")
print()

# Show different types of variables
input_vars = [v for v in basic_model.variables if v.startswith("digit_")]
control_vars = [
    v for v in basic_model.variables if v in ["num_addends", "num_digits", "template"]
]
output_vars = [v for v in basic_model.variables if v.startswith("output_digit_")]

print(f"Input variables ({len(input_vars)}): {input_vars}")
print(f"Control variables ({len(control_vars)}): {control_vars}")
print(f"Output variables ({len(output_vars)}): {output_vars}")
print("Special variables: raw_input, raw_output")
print()
print("Key insight: This is a direct input-output model")
print("Output digits are computed directly from input digits")
print("No explicit intermediate variables for sum or carry")

Basic Causal Model: addition_basic_2n_2d
Total variables: 12

Input variables (4): ['digit_0_0', 'digit_0_1', 'digit_1_0', 'digit_1_1']
Control variables (3): ['num_addends', 'num_digits', 'template']
Output variables (3): ['output_digit_0', 'output_digit_1', 'output_digit_2']
Special variables: raw_input, raw_output

Key insight: This is a direct input-output model
Output digits are computed directly from input digits
No explicit intermediate variables for sum or carry


### Sampling Valid Inputs

The model can sample valid addition problems:

In [32]:
# Sample a 2-digit addition problem
sample = sample_valid_addition_input(config_2n_2d, num_addends=2, num_digits=2)

print("Sampled Input:")
print(f"Number of addends: {sample['num_addends']}")
print(f"Digits per number: {sample['num_digits']}")
print()
print("Input digits:")
for k in range(2):
    digits = [sample[f"digit_{k}_0"], sample[f"digit_{k}_1"]]
    number = processor.digits_to_number(digits)
    print(f"  Number {k}: digits={digits}, value={number}")

Sampled Input:
Number of addends: 2
Digits per number: 2

Input digits:
  Number 0: digits=[1, 4], value=14
  Number 1: digits=[7, 3], value=73


### Running the Causal Model

Let's run the model to generate a prompt and answer:

In [33]:
# Run the causal model with our sample as an intervention
output = basic_model.run_forward(sample)

print("Causal Model Output:")
print(f"Generated prompt: '{output['raw_input']}'")
print(f"Expected answer: '{output['raw_output']}'")
print()

# Verify the answer is correct
num1 = processor.digits_to_number([sample["digit_0_0"], sample["digit_0_1"]])
num2 = processor.digits_to_number([sample["digit_1_0"], sample["digit_1_1"]])
expected_sum = num1 + num2

print(f"Verification: {num1} + {num2} = {expected_sum}")
print(f"Model output: '{output['raw_output'].strip()}'")
print(f"Correct: {str(expected_sum) == output['raw_output'].strip()}")

Causal Model Output:
Generated prompt: 'The sum of 14 and 73 is'
Expected answer: ' 87'

Verification: 14 + 73 = 87
Model output: '87'
Correct: True


### Multiple Examples

In [34]:
print("Multiple Example Generations:")
print("=" * 70)

for i in range(5):
    # Sample new input
    sample = sample_valid_addition_input(config_2n_2d, num_addends=2, num_digits=2)
    # Run model
    output = basic_model.run_forward(sample)

    # Extract numbers
    num1 = processor.digits_to_number([sample["digit_0_0"], sample["digit_0_1"]])
    num2 = processor.digits_to_number([sample["digit_1_0"], sample["digit_1_1"]])
    expected = num1 + num2

    print(f"\nExample {i + 1}:")
    print(f"  Prompt: '{output['raw_input']}'")
    print(f"  Answer: '{output['raw_output'].strip()}'")
    print(f"  Verification: {num1} + {num2} = {expected} ✓")

Multiple Example Generations:

Example 1:
  Prompt: 'The sum of 20 and 25 is'
  Answer: '45'
  Verification: 20 + 25 = 45 ✓

Example 2:
  Prompt: 'The sum of 53 and 93 is'
  Answer: '146'
  Verification: 53 + 93 = 146 ✓

Example 3:
  Prompt: 'The sum of 86 and 87 is'
  Answer: '173'
  Verification: 86 + 87 = 173 ✓

Example 4:
  Prompt: 'The sum of 22 and 79 is'
  Answer: '101'
  Verification: 22 + 79 = 101 ✓

Example 5:
  Prompt: 'The sum of 55 and 52 is'
  Answer: '107'
  Verification: 55 + 52 = 107 ✓


## Step 4: Intermediate Structure Causal Model

Now let's look at a more sophisticated model that explicitly represents **carry variables** at each digit position.

This model tests the hypothesis that neural networks might use explicit carry propagation when performing addition.

In [35]:
from causalab.tasks.general_addition.causal_models import create_intermediate_addition_model

### Understanding Intermediate Variables

The intermediate model includes:
- **C_i**: Carry from position i (1 = ones, 2 = tens, etc.)
  - C_1 = 1 if ones digits sum to ≥ 10, else 0
  - C_i = 1 if (digit_i + digit_i + C_{i-1}) ≥ 10, else 0
- **O_i**: Output digit at position i
  - O_1 = (ones digits) % 10
  - O_i = (digit_i + digit_i + C_{i-1}) % 10

**Important indexing**:
- Input digits use **left-to-right** indexing: digit_{k}_{d} where d=0 is most significant
- Intermediate/output use **right-to-left** indexing starting at 1: C_i, O_i where i=1 is ones place

This matches how addition works: we compute from right to left!

In [36]:
# Create intermediate model (only works for 2 addends)
intermediate_model = create_intermediate_addition_model(config_2n_2d)

print(f"Intermediate Causal Model: {intermediate_model.id}")
print(f"Total variables: {len(intermediate_model.variables)}")
print()

# Show intermediate variables
carry_vars = [v for v in intermediate_model.variables if v.startswith("C_")]
output_vars = [v for v in intermediate_model.variables if v.startswith("O_")]

print("Input variables (4): digit_0_0, digit_0_1, digit_1_0, digit_1_1")
print(f"Intermediate carry variables ({len(carry_vars)}): {carry_vars}")
print(f"Output variables ({len(output_vars)}): {output_vars}")
print()
print("Key insight: This model has explicit carry propagation")
print("We can test if neural networks use similar computational structure")

Intermediate Causal Model: addition_intermediate_2n_2d
Total variables: 13

Input variables (4): digit_0_0, digit_0_1, digit_1_0, digit_1_1
Intermediate carry variables (2): ['C_1', 'C_2']
Output variables (3): ['O_1', 'O_2', 'O_3']

Key insight: This model has explicit carry propagation
We can test if neural networks use similar computational structure


### Visualizing the Computation

Let's walk through an example step by step: 27 + 45 = 72

In [37]:
# Create specific example: 27 + 45
example_input = {
    "digit_0_0": 2,
    "digit_0_1": 7,  # 27
    "digit_1_0": 4,
    "digit_1_1": 5,  # 45
    "num_digits": 2,
    "template": config_2n_2d.templates[0],
}

# Run the intermediate model
intermediate_output = intermediate_model.run_forward(example_input)

print("Example: 27 + 45 = 72")
print("=" * 70)
print()
print("Step-by-step computation (right-to-left):")
print()
print("Position 1 (ones place):")
print("  Input digits: 7 + 5")
print(f"  C_1 = {intermediate_output['C_1']} (since 7 + 5 = 12 >= 10)")
print(f"  O_1 = {intermediate_output['O_1']} ((7 + 5) % 10 = 12 % 10)")
print()
print("Position 2 (tens place):")
print("  Input digits: 2 + 4")
print(f"  C_2 = {intermediate_output['C_2']} (since 2 + 4 + 1 = 7 < 10)")
print(f"  O_2 = {intermediate_output['O_2']} ((2 + 4 + C_1) % 10 = (2 + 4 + 1) % 10)")
print()
print("Position 3 (hundreds place - carry out):")
print(f"  O_3 = {intermediate_output['O_3']} (equals C_2)")
print()
print(f"Final answer: {intermediate_output['raw_output']}")
print()
print("Note: Position indexing starts at 1 (ones place), unlike arrays!")

Example: 27 + 45 = 72

Step-by-step computation (right-to-left):

Position 1 (ones place):
  Input digits: 7 + 5
  C_1 = 1 (since 7 + 5 = 12 >= 10)
  O_1 = 2 ((7 + 5) % 10 = 12 % 10)

Position 2 (tens place):
  Input digits: 2 + 4
  C_2 = 0 (since 2 + 4 + 1 = 7 < 10)
  O_2 = 7 ((2 + 4 + C_1) % 10 = (2 + 4 + 1) % 10)

Position 3 (hundreds place - carry out):
  O_3 = 0 (equals C_2)

Final answer:  72

Note: Position indexing starts at 1 (ones place), unlike arrays!


### Example with Carry Propagation: 58 + 67 = 125

In [38]:
# Example with multiple carries: 58 + 67 = 125
carry_example = {
    "digit_0_0": 5,
    "digit_0_1": 8,  # 58
    "digit_1_0": 6,
    "digit_1_1": 7,  # 67
    "num_digits": 2,
    "template": config_2n_2d.templates[0],
}

carry_output = intermediate_model.run_forward(carry_example)

print("Example with Carry Propagation: 58 + 67 = 125")
print("=" * 70)
print()
print("Position 1 (ones): 8 + 7 = 15")
print(f"  C_1 = {carry_output['C_1']} ✓ (generates carry)")
print(f"  O_1 = {carry_output['O_1']} (15 % 10)")
print()
print("Position 2 (tens): 5 + 6 = 11, plus C_1 = 1")
print(f"  C_2 = {carry_output['C_2']} ✓ (generates carry)")
print(f"  O_2 = {carry_output['O_2']} ((5 + 6 + 1) % 10 = 12 % 10)")
print()
print("Position 3 (hundreds): carry out")
print(f"  O_3 = {carry_output['O_3']} (equals C_2)")
print()
print(f"Final answer: {carry_output['raw_output']}")
print()
print("This shows carry propagation: ones → tens → hundreds")

Example with Carry Propagation: 58 + 67 = 125

Position 1 (ones): 8 + 7 = 15
  C_1 = 1 ✓ (generates carry)
  O_1 = 5 (15 % 10)

Position 2 (tens): 5 + 6 = 11, plus C_1 = 1
  C_2 = 1 ✓ (generates carry)
  O_2 = 2 ((5 + 6 + 1) % 10 = 12 % 10)

Position 3 (hundreds): carry out
  O_3 = 1 (equals C_2)

Final answer:  125

This shows carry propagation: ones → tens → hundreds


## Step 5: Comparing Basic vs Intermediate Models

Both models produce the same input/output behavior, but have different internal structure.

In [39]:
# Run both models on the same input
test_input = {
    "digit_0_0": 3,
    "digit_0_1": 4,  # 34
    "digit_1_0": 5,
    "digit_1_1": 9,  # 59
    "num_addends": 2,
    "num_digits": 2,
    "template": config_2n_2d.templates[0],
}

basic_result = basic_model.run_forward(test_input)
intermediate_result = intermediate_model.run_forward(test_input)

print("Comparison: 34 + 59 = 93")
print("=" * 70)
print()
print("Basic Model (direct computation):")
print("  Variables: input digits → output digits")
print(f"  Prompt: '{basic_result['raw_input']}'")
print(f"  Answer: '{basic_result['raw_output']}'")
print(f"  Output digits: {[basic_result[f'output_digit_{i}'] for i in range(3)]}")
print()
print("Intermediate Model (with carry variables):")
print("  Variables: input digits → C_i (carries) → O_i (outputs)")
print(f"  Prompt: '{intermediate_result['raw_input']}'")
print(f"  Answer: '{intermediate_result['raw_output']}'")
print("  Intermediate values:")
print(f"    C_1={intermediate_result['C_1']} → O_1={intermediate_result['O_1']}")
print(f"    C_2={intermediate_result['C_2']} → O_2={intermediate_result['O_2']}")
print(f"    O_3={intermediate_result['O_3']} (carry out)")
print()
print("Both models agree on input/output behavior: ✓")
print("But they represent different computational hypotheses!")

Comparison: 34 + 59 = 93

Basic Model (direct computation):
  Variables: input digits → output digits
  Prompt: 'The sum of 34 and 59 is'
  Answer: ' 93'
  Output digits: [0, 9, 3]

Intermediate Model (with carry variables):
  Variables: input digits → C_i (carries) → O_i (outputs)
  Prompt: 'The sum of 34 and 59 is'
  Answer: ' 93'
  Intermediate values:
    C_1=1 → O_1=3
    C_2=0 → O_2=9
    O_3=0 (carry out)

Both models agree on input/output behavior: ✓
But they represent different computational hypotheses!


## Step 6: Testing Interventions on Intermediate Variables

The power of the intermediate model is that we can intervene on carry variables.

In [40]:
# Original: 23 + 45 = 68 (no carry from ones)
input_no_carry = {
    "digit_0_0": 2,
    "digit_0_1": 3,
    "digit_1_0": 4,
    "digit_1_1": 5,
    "num_digits": 2,
    "template": config_2n_2d.templates[0],
}

output_original = intermediate_model.run_forward(input_no_carry)

print("Original computation: 23 + 45 = 68")
print(f"  C_1 = {output_original['C_1']} (3 + 5 = 8, no carry)")
print(f"  O_1 = {output_original['O_1']}")
print(f"  O_2 = {output_original['O_2']} (2 + 4 + 0)")
print(f"  Answer: {output_original['raw_output'].strip()}")
print()

# Intervene: Force C_1 = 1 (pretend ones place generated a carry)
output_intervened = intermediate_model.run_forward({**input_no_carry, "C_1": 1})

print("Intervention: Force C_1 = 1 (as if 3 + 5 >= 10)")
print(f"  C_1 = {output_intervened['C_1']} (intervened)")
print(f"  O_1 = {output_intervened['O_1']} (unchanged)")
print(f"  O_2 = {output_intervened['O_2']} (2 + 4 + 1 = 7, affected by carry!)")
print(f"  Answer: {output_intervened['raw_output'].strip()}")
print()
print("Key insight: Interventions on C_1 affect downstream variables!")
print("This lets us test if neural networks represent carries explicitly.")

Original computation: 23 + 45 = 68
  C_1 = 0 (3 + 5 = 8, no carry)
  O_1 = 8
  O_2 = 6 (2 + 4 + 0)
  Answer: 68

Intervention: Force C_1 = 1 (as if 3 + 5 >= 10)
  C_1 = 1 (intervened)
  O_1 = 8 (unchanged)
  O_2 = 7 (2 + 4 + 1 = 7, affected by carry!)
  Answer: 78

Key insight: Interventions on C_1 affect downstream variables!
This lets us test if neural networks represent carries explicitly.


## Step 7: Testing with Different Digit Counts

In [41]:
# Create 3-digit intermediate model
config_3d = create_two_number_three_digit_config()
model_3d = create_intermediate_addition_model(config_3d)

# Test with 3-digit numbers: 123 + 456 = 579
test_3d = {
    "digit_0_0": 1,
    "digit_0_1": 2,
    "digit_0_2": 3,  # 123
    "digit_1_0": 4,
    "digit_1_1": 5,
    "digit_1_2": 6,  # 456
    "num_digits": 3,
    "template": config_3d.templates[0],
}

result_3d = model_3d.run_forward(test_3d)

print("3-Digit Addition: 123 + 456 = 579")
print("=" * 70)
print(f"Prompt: '{result_3d['raw_input']}'")
print(f"Answer: '{result_3d['raw_output']}'")
print()
print("Intermediate computation:")
for i in range(1, 4):
    print(f"  Position {i}: C_{i}={result_3d[f'C_{i}']}, O_{i}={result_3d[f'O_{i}']}")
print(f"  Position 4 (carry out): O_4={result_3d['O_4']}")
print()
print("The same intermediate structure scales to more digits!")

3-Digit Addition: 123 + 456 = 579
Prompt: 'The sum of 123 and 456 is'
Answer: ' 579'

Intermediate computation:
  Position 1: C_1=0, O_1=9
  Position 2: C_2=0, O_2=7
  Position 3: C_3=0, O_3=5
  Position 4 (carry out): O_4=0

The same intermediate structure scales to more digits!


## What We've Built

This system provides a flexible foundation for addition experiments:

### Core Components:
1. **Configuration**: Handle K numbers with D digits each
2. **Template System**: Convert digits to formatted prompts
3. **Two Causal Models**:
   - **Basic**: Direct input-output mapping
   - **Intermediate**: Explicit carry variables (C_i) and output variables (O_i)

### Key Design Decisions:
- **Scalable**: Same framework for 2-digit, 3-digit, 4-digit, etc.
- **Two Hypotheses**: Test whether models use carry propagation
- **Flexible**: Easy to extend to more addends or different templates
- **1-Indexed Positions**: C_i and O_i start at 1 (ones place), matching mathematical convention

### The Core Research Question:
When neural networks perform addition, do they:
1. **Learn a direct mapping** (like the basic model)?
2. **Use explicit carry propagation** (like the intermediate model with C_i variables)?

We can test this with causal interventions!

### Next Steps:
1. **Token Position Functions**: Locate digits in neural network inputs
2. **Counterfactual Generation**: Create test cases for interventions
3. **Neural Experiments**: Run interchange interventions to test hypotheses

### Variable Naming Convention:
- **Inputs**: digit_{k}_{d} where k=number index, d=digit position (0=leftmost)
- **Carries**: C_i where i=position from right (1=ones, 2=tens, ...)
- **Outputs**: O_i where i=position from right (1=ones, 2=tens, ...)
- This mixed indexing reflects both input format (left-to-right) and computation order (right-to-left)